In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlopen
from bs4 import BeautifulSoup

config = json.load(open('../config.json'))
APP_ID = config['yahoo']
pageurl = "http://jlp.yahooapis.jp/MAService/V1/parse"

形態素解析器の作成

In [2]:
def get_words(sentence, appid=APP_ID, results="ma", filter="1|2|3|4|5|9|10"):
    sentence = sentence.encode('utf8')
    params = urlencode({
            'appid': appid,
            'results': results,
            'filter': filter,
            'sentence': sentence
        })
    results = urlopen(pageurl, params.encode('utf8'))
    soup = BeautifulSoup(results.read())
    return [w.surface.string for w in soup.ma_result.word_list]
print(get_words(u'今日のご飯は何かな'))

['今日', 'ご飯', '何']


ナイーブベイズモデルの作成

In [18]:
import math
class NaiveBayes(object):
    def __init__(self):
        self.vocabularies = set()
        self.wordcount = {}  # {cat: {word: n, ...}, ...}
        self.catcount = {}  # {cat: n, ...}

    def train(self, doc, cat):
        words = get_words(doc)
        self.catcount.setdefault(cat, 0)
        self.catcount[cat] += 1
        for word in words:
            self.vocabularies.add(word)
            self.wordcount.setdefault(cat, {})
            self.wordcount[cat].setdefault(word, 0)
            self.wordcount[cat][word] += 1
    
    def prior(self, cat):
        return 1. * self.catcount[cat] / sum(self.catcount.values())
    
    def wordprob(self, word, cat):
        n_word = self.incategory(word, cat) + 1.
        n_cat = sum(self.wordcount[cat].values()) + len(self.vocabularies)
        return 1. * n_word / n_cat
        
    def incategory(self, word, cat):
        if word in self.wordcount[cat]:
            return 1. * self.wordcount[cat][word]
        else:
            return 0.

    def score(self, words, cat):
        score = math.log(self.prior(cat))
        for word in words:
            score += math.log(self.wordprob(word, cat))
        return score
    
    def classifier(self, doc):
        max_prob = -1000
        best_cat = None
        words = get_words(doc)
        for cat in self.catcount.keys():
            prob = self.score(words, cat)
            if prob > max_prob:
                max_prob = prob
                best_cat = cat
        return best_cat

In [19]:
nb = NaiveBayes()
nb.train(u'''Python（パイソン）は，オランダ人のグイド・ヴァンロッサムが作ったオープンソースのプログラミング言語。
オブジェクト指向スクリプト言語の一種であり，Perlとともに欧米で広く普及している。イギリスのテレビ局 BBC が製作した
コメディ番組『空飛ぶモンティパイソン』にちなんで名付けられた。Python は英語で爬虫類のニシキヘビの意味で，
Python言語のマスコットやアイコンとして使われることがある。Pythonは汎用の高水準言語である。
プログラマの生産性とコードの信頼性を重視して設計されており，核となるシンタックスおよびセマンティクスは必要最小限に抑えられている反面，
利便性の高い大規模な標準ライブラリを備えている。
Unicode による文字列操作をサポートしており，日本語処理も標準で可能である。
多くのプラットフォームをサポートしており（動作するプラットフォーム），また，豊富なドキュメント，豊富なライブラリがあることから，
産業界でも利用が増えつつある。''', 'Python')

nb.train(u'''Ruby（ルビー）は，まつもとゆきひろ（通称Matz）により開発されたオブジェクト指向スクリプト言語であり，
従来 Perlなどのスクリプト言語が用いられてきた領域でのオブジェクト指向プログラミングを実現する。Rubyは当初1993年2月24日に生まれ， 
1995年12月にfj上で発表された。名称のRubyは，プログラミング言語Perlが6月の誕生石であるPearl（真珠）と同じ発音をすることから，
まつもとの同僚の誕生石（7月）のルビーを取って名付けられた。''', 'Ruby')

nb.train(u'''豊富な機械学習（きかいがくしゅう，Machine learning）とは，人工知能における研究課題の一つで，
人間が自然に行っている学習能力と同様の機能をコンピュータで実現させるための技術・手法のことである。
ある程度の数のサンプルデータ集合を対象に解析を行い，そのデータから有用な規則，ルール，知識表現，判断基準などを抽出する。
データ集合を解析するため，統計学との関連も非常に深い。
機械学習は検索エンジン，医療診断，スパムメールの検出，金融市場の予測，DNA配列の分類，音声認識や文字認識などのパターン認識，
ゲーム戦略，ロボット，など幅広い分野で用いられている。応用分野の特性に応じて学習手法も適切に選択する必要があり，
様々な手法が提案されている。それらの手法は， Machine Learning や IEEE Transactions on Pattern Analysis
and Machine Intelligence などの学術雑誌などで発表されることが多い。''', u'機械学習')

In [20]:
words = u'Python はオランダ生まれって知ってました？'
print(u'%s => 推定カテゴリ: %s' % (words ,nb.classifier(words)))

Python はオランダ生まれって知ってました？ => 推定カテゴリ: Python


In [21]:
words = u'まつもとさんって人が作ったプログラミング言語'
print(u'%s => 推定カテゴリ: %s' % (words ,nb.classifier(words)))

まつもとさんって人が作ったプログラミング言語 => 推定カテゴリ: Ruby


In [22]:
words = u'機械学習は様々な分野に使われ始めています。'
print(u'%s => 推定カテゴリ: %s' % (words ,nb.classifier(words)))

機械学習は様々な分野に使われ始めています。 => 推定カテゴリ: 機械学習


Oreore Unit testing

In [66]:
def test_train(nb):
    print('vocabularies:', nb.vocabularies == set(['機械', '学習', 'へび', '言語']))
    print('wordcount:', nb.wordcount == {'ml': {'機械': 1, '学習': 1}, 'python': {'へび': 1, '言語': 2}})
    print('catcount:', nb.catcount == {'ml': 1, 'python': 1})
    
def test_prior(nb):
    print('prior:', nb.prior('python') == 0.5)

def test_wordprob(nb):
    print('wordprob included:', nb.wordprob('機械', 'ml') == 1. * (1 + 1) / (2 + 4))
    print('wordprob nonincluded:', nb.wordprob('SVM', 'ml') == 1. * 1 / (2 + 4))
    
def test_incategory(nb):
    print('incategory:', nb.incategory('機械', 'ml') == 1)
        
nb = NaiveBayes()
nb.train('機械と学習', 'ml')
nb.train('へびと言語言語', 'python')
test_train(nb)
test_prior(nb)
test_wordprob(nb)
test_incategory(nb)

vocabularies: True
wordcount: True
catcount: True
prior: True
wordprob included: True
wordprob nonincluded: True
incategory: True
